<a href="https://colab.research.google.com/github/kshitij-sinha019/PLAGIRISM_CHECKER/blob/main/PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn pandas numpy torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!ls /content/drive/MyDrive/kaggle


kaggle.json  mit-plagiarism-detection-dataset.zip  train_snli.txt


In [23]:
!unzip "/content/drive/MyDrive/kaggle/mit-plagiarism-detection-dataset.zip" -d /content/plagiarism_dataset


Archive:  /content/drive/MyDrive/kaggle/mit-plagiarism-detection-dataset.zip
  inflating: /content/plagiarism_dataset/train_snli.txt  


In [24]:
!head /content/plagiarism_dataset/train_snli.txt


A person on a horse jumps over a broken down airplane.	A person is at a diner, ordering an omelette.	0
A person on a horse jumps over a broken down airplane.	A person is outdoors, on a horse.	1
Children smiling and waving at camera	There are children present	1
Children smiling and waving at camera	The kids are frowning	0
A boy is jumping on skateboard in the middle of a red bridge.	The boy skates down the sidewalk.	0
A boy is jumping on skateboard in the middle of a red bridge.	The boy does a skateboarding trick.	1
An older man sits with his orange juice at a small table in a coffee shop while employees in bright colored shirts smile in the background.	A boy flips a burger.	0
Two blond women are hugging one another.	The women are sleeping.	0
Two blond women are hugging one another.	There are women showing affection.	1
A few people in a restaurant setting, one of them is drinking orange juice.	The people are sitting at desks in school.	0


In [106]:
import pandas as pd


data_path = "/content/plagiarism_dataset/train_snli.txt"
df = pd.read_csv(data_path, sep="\t", header=None, names=["Sentence1", "Sentence2", "Label"])


df.head()


,Sentence1,Sentence2,Label
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [107]:
import re
import nltk
from nltk.corpus import stopwords


nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text

df["Sentence1"] = df["Sentence1"].apply(clean_text)
df["Sentence2"] = df["Sentence2"].apply(clean_text)

df.head()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Sentence1,Sentence2,Label
0,person horse jumps broken airplane,person diner ordering omelette,0
1,person horse jumps broken airplane,person outdoors horse,1
2,children smiling waving camera,children present,1
3,children smiling waving camera,kids frowning,0
4,boy jumping skateboard middle red bridge,boy skates sidewalk,0


In [91]:

!pip install scikit-learn pandas numpy nltk


import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer


nltk.download("stopwords")
nltk.download("punkt")


from google.colab import drive
drive.mount('/content/drive')


data_path = "/content/drive/MyDrive/kaggle/train_snli.txt"
df = pd.read_csv(data_path, sep="\t")


if df.columns[0] != "Sentence1":
    df.columns = ["Sentence1", "Sentence2", "Label"]


tokenizer = TreebankWordTokenizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

custom_stopwords = {"would", "could", "also", "one", "get", "us"}


def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    tokens = tokenizer.tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word not in custom_stopwords]
    return " ".join(tokens)


df["Sentence1"] = df["Sentence1"].apply(clean_text)
df["Sentence2"] = df["Sentence2"].apply(clean_text)


df.dropna(subset=["Label"], inplace=True)


df["Combined"] = df["Sentence1"] + " " + df["Sentence2"]


strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in strat_split.split(df["Combined"], df["Label"]):
    X_train, X_test = df["Combined"].iloc[train_index], df["Combined"].iloc[test_index]
    y_train, y_test = df["Label"].iloc[train_index], df["Label"].iloc[test_index]


vectorizer = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


model = SGDClassifier(loss="hinge", penalty="l2", alpha=0.00005, max_iter=1500, tol=1e-4, random_state=42)
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)


accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized SLM Model Accuracy: {accuracy * 100:.2f}%")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Optimized SLM Model Accuracy: 65.11%


In [86]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 3000]
}

grid_search = GridSearchCV(SGDClassifier(), param_grid, cv=5, verbose=1)
grid_search.fit(X_train_tfidf, y_train)

print("Best Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'alpha': 0.0001, 'max_iter': 2000}


In [88]:
from sklearn.linear_model import SGDClassifier


best_model = SGDClassifier(alpha=0.0001, max_iter=2000)


best_model.fit(X_train_tfidf, y_train)


SGDClassifier(max_iter=2000)

In [89]:
from sklearn.metrics import classification_report


y_pred = best_model.predict(X_test_tfidf)


print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.64      0.65      0.65     36793
           1       0.65      0.63      0.64     36682

    accuracy                           0.64     73475
   macro avg       0.64      0.64      0.64     73475
weighted avg       0.64      0.64      0.64     73475



In [92]:
import os


data_path = '/content/drive/MyDrive/kaggle/mit-plagiarism-detection/train_snli.txt'


if os.path.exists(data_path):
    print(f"File found at: {data_path}")
    df = pd.read_csv(data_path, sep="\t")
    print(df.head())
else:
    print(f"File not found at: {data_path}. Please check the path and file name.")


File found at: /content/drive/MyDrive/kaggle/mit-plagiarism-detection/train_snli.txt
  A person on a horse jumps over a broken down airplane.  \
0  A person on a horse jumps over a broken down a...       
1              Children smiling and waving at camera       
2              Children smiling and waving at camera       
3  A boy is jumping on skateboard in the middle o...       
4  A boy is jumping on skateboard in the middle o...       

  A person is at a diner, ordering an omelette.  0  
0             A person is outdoors, on a horse.  1  
1                    There are children present  1  
2                         The kids are frowning  0  
3             The boy skates down the sidewalk.  0  
4           The boy does a skateboarding trick.  1  


In [93]:
import joblib

joblib.dump(model, '/content/drive/MyDrive/kaggle/optimized_svm_model.pkl')

joblib.dump(vectorizer, '/content/drive/MyDrive/kaggle/tfidf_vectorizer.pkl')


['/content/drive/MyDrive/kaggle/tfidf_vectorizer.pkl']

In [94]:
import joblib
import pandas as pd


model = joblib.load('/content/drive/MyDrive/kaggle/optimized_svm_model.pkl')
vectorizer = joblib.load('/content/drive/MyDrive/kaggle/tfidf_vectorizer.pkl')

new_data = [
    ["A person is riding a horse.", "A man is on a horse."],
    ["Children are playing in the park.", "There are kids playing soccer."]
]


df_new = pd.DataFrame(new_data, columns=["Sentence1", "Sentence2"])


df_new["Combined"] = df_new["Sentence1"] + " " + df_new["Sentence2"]


X_new_tfidf = vectorizer.transform(df_new["Combined"])


predictions = model.predict(X_new_tfidf)


for i, (sentence_pair, prediction) in enumerate(zip(new_data, predictions)):
    print(f"Sentence Pair {i+1}: {sentence_pair}")
    print(f"Prediction: {'Same' if prediction == 1 else 'Different'}\n")


Sentence Pair 1: ['A person is riding a horse.', 'A man is on a horse.']
Prediction: Same

Sentence Pair 2: ['Children are playing in the park.', 'There are kids playing soccer.']
Prediction: Different



In [95]:
from sklearn.metrics import classification_report


print("Classification Report on Test Data:")
print(classification_report(y_test, y_pred))


Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.66      0.63      0.64     36793
           1       0.64      0.67      0.66     36682

    accuracy                           0.65     73475
   macro avg       0.65      0.65      0.65     73475
weighted avg       0.65      0.65      0.65     73475



In [96]:
import joblib


model = joblib.load('/content/drive/MyDrive/kaggle/optimized_svm_model.pkl')
vectorizer = joblib.load('/content/drive/MyDrive/kaggle/tfidf_vectorizer.pkl')


In [97]:
def predict_sentence_pair(sentence1, sentence2):

    combined = [sentence1 + " " + sentence2]
    transformed_input = vectorizer.transform(combined)


    prediction = model.predict(transformed_input)

    return "Same" if prediction == 1 else "Different"


print(predict_sentence_pair("The dog is playing in the yard.", "A dog is running around the yard."))


Different
